In [1]:
import sys
print(sys.executable)


d:\ML_PROJECT\time_series_analysis\venv\Scripts\python.exe


In [5]:
#libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use("dark_background")
from pandas_datareader.data import DataReader
import yfinance as yf
from datetime import datetime


In [ ]:
#available stock names list
url= "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table =pd.read_html(url)[0] # reads the first table
codes =table["Symbol"].tolist()
names=table["Security"].tolist()
list(zip(codes[:100],names[:100]))

[('MMM', '3M'),
 ('AOS', 'A. O. Smith'),
 ('ABT', 'Abbott Laboratories'),
 ('ABBV', 'AbbVie'),
 ('ACN', 'Accenture'),
 ('ADBE', 'Adobe Inc.'),
 ('AMD', 'Advanced Micro Devices'),
 ('AES', 'AES Corporation'),
 ('AFL', 'Aflac'),
 ('A', 'Agilent Technologies'),
 ('APD', 'Air Products'),
 ('ABNB', 'Airbnb'),
 ('AKAM', 'Akamai Technologies'),
 ('ALB', 'Albemarle Corporation'),
 ('ARE', 'Alexandria Real Estate Equities'),
 ('ALGN', 'Align Technology'),
 ('ALLE', 'Allegion'),
 ('LNT', 'Alliant Energy'),
 ('ALL', 'Allstate'),
 ('GOOGL', 'Alphabet Inc. (Class A)'),
 ('GOOG', 'Alphabet Inc. (Class C)'),
 ('MO', 'Altria'),
 ('AMZN', 'Amazon'),
 ('AMCR', 'Amcor'),
 ('AEE', 'Ameren'),
 ('AEP', 'American Electric Power'),
 ('AXP', 'American Express'),
 ('AIG', 'American International Group'),
 ('AMT', 'American Tower'),
 ('AWK', 'American Water Works'),
 ('AMP', 'Ameriprise Financial'),
 ('AME', 'Ametek'),
 ('AMGN', 'Amgen'),
 ('APH', 'Amphenol'),
 ('ADI', 'Analog Devices'),
 ('ANSS', 'Ansys'),
 ('A

In [12]:
# the companies we r gonna be analysing
tech_list=['AMZN','GOOG','ACN','MMM']
end= datetime.now()
start=datetime(end.year-1,end.month,end.day)
stock_data={}
for stock in tech_list:
    stock_data[stock]=yf.download(stock,start,end)
company_list=list(stock_data.values())
company_name =["AMAZON","GOOGLE","ACCENTURE","3M"]



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
